In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from tqdm import tqdm
# Load the data
df_news = pd.read_csv('/Users/macbook/Documents/PhD_Documents/embedding_methods/news_data/news_total_file.csv')

# Drop 'Unnamed: 0' column and remove duplicate values
df_news = df_news.drop(columns=['Unnamed: 0'], errors='ignore').drop_duplicates()

# Ensure the Date column is in datetime format
df_news['Date'] = pd.to_datetime(df_news['Date'], errors='coerce')

# Drop rows where the Date conversion failed
df_news = df_news.dropna(subset=['Date'])

# Sort by Date and format it as YYYY-MM-DD
df_news = df_news.sort_values(by='Date')
df_news['Date'] = df_news['Date'].dt.strftime('%Y-%m-%d')

# Drop duplicates again after formatting Date
df_news = df_news.drop_duplicates()

# Display the cleaned DataFrame
print(df_news)

/var/folders/jw/_prd4c6n32ggl_4pwszrqnsh0000gn/T/ipykernel_26070/3764355448.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_news['Date'] = pd.to_datetime(df_news['Date'], errors='coerce')


              Date                                               Text
1328    2014-01-01  railcar north dakota crude train crash older l...
1329    2014-01-01  skorea dec crude oil import 84 pct yy prelimin...
1330    2014-01-01  updat 1iraq oil export averag 2341 mln bpd dec...
1331    2014-01-01  iraq oil export averag 2341 mln bpd decemb min...
1332    2014-01-01  brazil petrobra start product roncador field p...
...            ...                                                ...
592597  2024-01-31  us senat committe energi natur resourc full co...
592596  2024-01-31  new jersey natur ga compani new jersey natur g...
592595  2024-01-31  oil lower us crude stock build score first mon...
592665  2024-01-31                    oil price goe brent 8219 barrel
592857  2024-01-31  russia novak current oil price adequ reflect m...

[592858 rows x 2 columns]


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/macbook/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

In [4]:
def get_embedding(word, embeddings_index):
    return embeddings_index.get(word)

In [5]:
def aggregate_embeddings(embeddings):
    valid_embeddings = [e for e in embeddings if e is not None]
    if not valid_embeddings:
        return np.zeros(300)  # Or use the dimensionality of your GloVe embeddings
    return np.mean(valid_embeddings, axis=0)

In [6]:
glove_embeddings = load_glove_embeddings('/Users/macbook/Documents/PhD_Documents/Second_paper/Glove/glove.6B/glove.6B.300d.txt')

In [7]:
# Initialize tqdm for pandas
tqdm.pandas()

# Tokenize the text with progress bar
df_news['tokens'] = df_news['Text'].progress_apply(word_tokenize)

# Convert tokens to embeddings with progress bar
df_news['embeddings'] = df_news['tokens'].progress_apply(lambda tokens: [get_embedding(token, glove_embeddings) for token in tokens])

# Aggregate the embeddings with progress bar
df_news['aggregated_embeddings'] = df_news['embeddings'].progress_apply(aggregate_embeddings)

print(df_news[['Text', 'aggregated_embeddings']])

100%|███████████████████████████████| 592858/592858 [00:05<00:00, 115135.18it/s]


                                                     Text  \
1328    railcar north dakota crude train crash older l...   
1329    skorea dec crude oil import 84 pct yy prelimin...   
1330    updat 1iraq oil export averag 2341 mln bpd dec...   
1331    iraq oil export averag 2341 mln bpd decemb min...   
1332    brazil petrobra start product roncador field p...   
...                                                   ...   
592597  us senat committe energi natur resourc full co...   
592596  new jersey natur ga compani new jersey natur g...   
592595  oil lower us crude stock build score first mon...   
592665                    oil price goe brent 8219 barrel   
592857  russia novak current oil price adequ reflect m...   

                                    aggregated_embeddings  
1328    [-0.22019853, 0.008930773, -0.008385662, -0.17...  
1329    [-0.15965118, 0.12035738, -0.08457201, 0.04844...  
1330    [-0.04776, 0.333462, -0.46915022, 0.26481092, ...  
1331    [-0.030609002, 0.30

In [9]:
embedding_dim = len(df_news['aggregated_embeddings'][0])  # Determine the embedding dimension

for i in range(embedding_dim):
    df_news[f'embedding_{i}'] = df_news['aggregated_embeddings'].apply(lambda x: x[i])

/var/folders/jw/_prd4c6n32ggl_4pwszrqnsh0000gn/T/ipykernel_26070/640309162.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_news[f'embedding_{i}'] = df_news['aggregated_embeddings'].apply(lambda x: x[i])
/var/folders/jw/_prd4c6n32ggl_4pwszrqnsh0000gn/T/ipykernel_26070/640309162.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_news[f'embedding_{i}'] = df_news['aggregated_embeddings'].apply(lambda x: x[i])
/var/folders/jw/_prd4c6n32ggl_4pwszrqnsh0000gn/T/ipykernel_26070/640309162.py:4: PerformanceWarning: DataFrame is 

In [10]:
df_news = df_news.drop(['tokens', 'embeddings', 'aggregated_embeddings'], axis = 1)

In [11]:
df_news = df_news.drop_duplicates()
df_news

,Date,Text,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_290,embedding_291,embedding_292,embedding_293,embedding_294,embedding_295,embedding_296,embedding_297,embedding_298,embedding_299
1328,2014-01-01,railcar north dakota crude train crash older l...,-0.220199,0.008931,-0.008386,-0.175972,-0.207096,0.146136,0.119886,0.121913,...,0.201564,-0.312289,0.130934,0.019791,0.066618,0.352566,-0.166862,-0.251981,0.063849,0.175074
1329,2014-01-01,skorea dec crude oil import 84 pct yy prelimin...,-0.159651,0.120357,-0.084572,0.048447,-0.317612,0.151685,0.131222,0.066929,...,-0.102906,-0.212680,-0.000646,-0.120956,0.210322,0.580534,-0.193662,-0.454552,-0.022702,0.084082
1330,2014-01-01,updat 1iraq oil export averag 2341 mln bpd dec...,-0.047760,0.333462,-0.469150,0.264811,-0.223658,0.197606,0.096934,0.295726,...,-0.686492,-0.047050,0.279440,0.085050,0.388484,0.590900,-0.299372,-0.367474,0.018488,0.216584
1331,2014-01-01,iraq oil export averag 2341 mln bpd decemb min...,-0.030609,0.301785,-0.308365,0.183584,-0.189015,0.265445,-0.020383,0.419155,...,-0.616537,-0.087877,0.318537,0.196048,0.264163,0.630777,-0.215228,-0.403250,-0.195443,0.164212
1332,2014-01-01,brazil petrobra start product roncador field p...,0.158301,0.493925,-0.205979,-0.122407,-0.162681,0.070380,-0.389218,0.205332,...,-0.088953,0.064245,-0.254516,0.118931,0.067867,0.304765,-0.047225,-0.275138,0.161017,0.075502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592597,2024-01-31,us senat committe energi natur resourc full co...,0.136721,-0.084791,-0.108012,-0.025767,-0.074842,-0.007351,0.097329,0.158474,...,-0.057648,0.047626,0.089715,-0.244262,0.161623,0.027615,-0.081114,-0.111262,0.052912,0.176354
592596,2024-01-31,new jersey natur ga compani new jersey natur g...,-0.203475,-0.271677,0.103724,-0.017313,-0.008392,-0.085105,0.176223,-0.316556,...,0.102120,-0.268364,-0.308526,-0.045701,-0.012913,0.221267,0.046027,-0.243642,0.090078,0.196936
592595,2024-01-31,oil lower us crude stock build score first mon...,-0.109164,0.135751,-0.072498,0.059030,-0.056838,0.085489,-0.016037,-0.004387,...,-0.186266,-0.306423,0.046540,0.106538,0.281406,0.464084,-0.198303,-0.535657,0.058442,0.002808
592665,2024-01-31,oil price goe brent 8219 barrel,-0.086152,0.168629,0.017906,0.274021,0.074130,0.369458,0.040070,0.064089,...,-0.176078,-0.233658,0.231339,0.149518,0.346016,0.519494,-0.287893,-0.411274,-0.236578,0.214821


In [12]:
df_news.to_csv('GLove_total_embedding.csv')

In [14]:
# Drop the 'Text' column
final_df = df_news.drop(columns=['Text'])

# Compute the mean of features grouped by the 'Date' column
mean_grouped_df = final_df.groupby('Date').mean()

# Display the resulting dataframe
print(mean_grouped_df)

            embedding_0  embedding_1  embedding_2  embedding_3  embedding_4  \
Date                                                                          
2014-01-01    -0.038010     0.234342    -0.176987     0.032436    -0.201255   
2014-01-02     0.020395     0.086490    -0.065630    -0.020100    -0.122848   
2014-01-03     0.035756     0.039905    -0.072978     0.004464    -0.071775   
2014-01-04     0.001827    -0.001946    -0.108958    -0.088731    -0.090556   
2014-01-05     0.009161     0.094795    -0.060559     0.028427    -0.059498   
...                 ...          ...          ...          ...          ...   
2024-01-27     0.038834     0.090737    -0.042215     0.041086    -0.101479   
2024-01-28     0.011447     0.081216    -0.037585    -0.037776    -0.092036   
2024-01-29    -0.013591     0.068397    -0.033324     0.031838    -0.047262   
2024-01-30    -0.003017     0.072778    -0.027889     0.032792    -0.055660   
2024-01-31    -0.017632     0.053241    -0.030460   

In [15]:
mean_grouped_df.to_csv('GLove_mean_embedding.csv')